In [ ]:
project = 'saga-vegvar-prod-znny'
use_colab_auth = True

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = ''

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

Denne spørringen regner ut gjennomsnittlig, minimum- og maksimumtemperatur per målestasjon og dato, fra 1. til 7. januar 2022.

Merk at den bruker både `maaledata` og `maalestasjoner`.

In [ ]:
query = f"""
SELECT
    navn,
    DATE(md.maaletidspunkt, "Europe/Oslo") dato,
    ROUND(AVG(lufttemperaturCelsius), 1) gjennomsnittLufttemperaturCelsius,
    MIN(lufttemperaturCelsius) minimumLufttemperaturCelsius,
    MAX(lufttemperaturCelsius) maksimumLufttemperaturCelsius

FROM `{project}.standardized.maaledata` md
JOIN `{project}.standardized.maalestasjoner` ms
    ON ms.id = md.maalestasjonId
    AND ms.versjon = md.maalestasjonVersjon
WHERE DATE(md.maaletidspunkt, "Europe/Oslo") BETWEEN "2022-01-01" AND "2022-01-07"
GROUP BY navn, dato
ORDER BY navn, dato
LIMIT 30
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen viser navn og koordinater for de 10 varmeste målestasjonene 1. januar 2022.

Merk at den bruker både `maaledata` og `maalestasjoner`.

In [ ]:
query = f"""
SELECT
    ANY_VALUE(navn) navn,
    ANY_VALUE(lokasjon) lokasjon,
    MAX(lufttemperaturCelsius) maksTemperatur,
    DATE(maaletidspunkt, "Europe/Oslo") dato
FROM `{project}.standardized.maaledata` md
JOIN `{project}.standardized.maalestasjoner` ms
    ON md.maalestasjonId = ms.id AND md.maalestasjonVersjon = ms.versjon

WHERE DATE(maaletidspunkt, "Europe/Oslo") = "2022-01-01"
GROUP BY ms.id, ms.versjon, dato
ORDER BY maksTemperatur DESC
LIMIT 10
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen henter id, versjon, navn og lokasjon for siste versjon av alle målestasjoner.

In [ ]:
query = f"""
SELECT * EXCEPT (row_number) FROM (
    SELECT id, versjon, navn, lokasjon, row_number() OVER (
        PARTITION BY id
        ORDER BY CAST(versjon AS INT) DESC
    ) row_number
    FROM `{project}.standardized.maalestasjoner`
)
WHERE row_number = 1
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen henter alle målestasjoner som har måledata fra det siste døgnet, sortert på målestasjonens navn.

In [ ]:
query = f"""
SELECT id, versjon, ANY_VALUE(navn) navn, ANY_VALUE(lokasjon) lokasjon
FROM `{project}.standardized.maalestasjoner` ms
JOIN `{project}.standardized.maaledata` md
    ON id = maalestasjonId AND versjon = maalestasjonVersjon
WHERE md.maaletidspunkt >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 24 HOUR)
GROUP BY id, versjon
ORDER BY navn
"""

print(query)

client.query(query).to_dataframe()